<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/elemi10-7506-Trabajo-Practico-2/Equipo_Kung_Fu_Panda_TP2_7506_Org_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TP2 - Real or Not?**

###  **Análisis Predictivo: Modelos Machine Learning**
<h4><a href = 'https://github.com/elemi10/7506-TP-Org-de-datos/tree/elemi10-7506-Trabajo-Practico-2'>github.com/elemi10/7506-TP-Org-de-datos</a></h4></center>

![Banner_TP2.png](https://drive.google.com/uc?id=1Q2Xz3ug_gUoqGnTF7TZ7Kx7hJz535wnp)




### Autenticamos y autorizamos conexión al Drive para  levantar archivos

In [21]:
# Código para leer archivo .csv y correr en google colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [22]:
# Autenticación de usuario
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [23]:
# Llamada de los archivos desde el Drive, usando file-ID
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

downloaded1 = drive.CreateFile({'id':'17pAgG9oJRK1bAFWRKkp96__zicG6yUmy'}) # replace the id with id of file you want to access
downloaded1.GetContentFile('test.csv') 

downloaded2 = drive.CreateFile({'id':'1u8v51BT7FZggIRD-eo0dQno--0wlxIhA'}) # replace the id with id of file you want to access
downloaded2.GetContentFile('sample_submission.csv') 

downloaded3 = drive.CreateFile({'id':'1ctj-5QfPnYNA2vNkSDrOf1oCk0MAEAo8'}) # replace the id with id of file you want to access
downloaded3.GetContentFile('df_train_preprocesado.xlsx') 


### Importamos **Librerías**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download("popular")
from sklearn import model_selection
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import HashingVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
from nltk.stem import PorterStemmer
from textblob import TextBlob
import xgboost as xgb
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
!pip install bert-tensorflow
import bert

### Levantamos los archivos para comenzar a trabajar en los modelos

*   **train.csv**
*   **test.csv**
*   **sample_submission.csv**





In [10]:
# train
train=pd.read_csv(r'train.csv', usecols=['keyword','location','text','target'])
train.tail()

,keyword,location,text,target
7608,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [15]:
# test
test=pd.read_csv(r'test.csv', usecols=['keyword','location','text'])
test.tail()

,keyword,location,text
3258,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...
3262,NaN,NaN,#CityofCalgary has activated its Municipal Eme...


In [16]:
# sample_submission
sample_submission=pd.read_csv(r"sample_submission.csv")
sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [25]:
# Categorizamos manualmente las keywords y creamos un nuevo archivo para despues utilizarlo en el DF original
categorias = pd.read_excel(r"df_train_preprocesado.xlsx", index_col=0, header=0, \
                           sheet_name="categorias_keyword_estadisticas",usecols=['keyword','Tipo_Desastre'])

In [26]:
categorias.head()

,Tipo_Desastre
keyword,
NaN,Miscelaneo
ablaze,Incendios
accident,Accidente/Riesgo
aftershock,Movimiento_de_Masa
airplane%20accident,Accidente/Riesgo


In [27]:
# DF limpio → para que se crea la columna?
categorias=categorias.iloc[:-1,:]

In [28]:
categorias.head()

,Tipo_Desastre
keyword,
NaN,Miscelaneo
ablaze,Incendios
accident,Accidente/Riesgo
aftershock,Movimiento_de_Masa
airplane%20accident,Accidente/Riesgo


In [29]:
# Hacemos un merge y agregamos nuevas columnas al DF
train=train.merge(categorias,how='outer',left_on='keyword',right_on='keyword')
test=test.merge(categorias,how='outer',left_on='keyword',right_on='keyword')
train.tail()

,keyword,location,text,target,Tipo_Desastre
7608,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0,Accidente/Riesgo
7609,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,Accidente/Riesgo
7610,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,Accidente/Riesgo
7611,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0,Accidente/Riesgo
7612,wrecked,NaN,Cramer: Iger's 3 words that wrecked Disney's s...,0,Accidente/Riesgo
